In [ ]:
import pandas as pd
import geopandas as gpd
import folium
import numpy as np
import matplotlib.pyplot as plt
import math
import matplotlib.colors as mcolors
import matplotlib.font_manager as font_manager
from sklearn import preprocessing
from math import pi
import shapely.geometry
import plotly.graph_objects as go

In [ ]:
# Read in data
data = pd.read_csv("../data/curated/NNpredi.csv")

# Calculate average rate by postcode and sort the top 10
data = data.groupby(by = ['Postcode']).mean()
data = data.sort_values(by = ['rate_2023'], ascending = False)
data = data.head(10)

In [ ]:
# Change postcode column from index to feature columns
postcode = [3478, 3318, 3480, 3540, 3755, 3759, 3260, 3273, 3987, 3757]
data.insert(0, "Postcode", postcode)
data.index = [1,2,3,4,5,6,7,8,9,10]

In [ ]:
# read the shape file
sf = gpd.read_file("../data/raw/POA_2021_AUST_GDA94.shp")
sf = sf.iloc[672:1366]

# Convert the geometry shaape to to latitude and longitude
sf['geometry'] = sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
sf = sf.rename(columns = {'POA_CODE21': 'Postcode'})

# create a JSON 
geoJSON = sf[['Postcode', 'geometry']].drop_duplicates('Postcode').to_json()


In [ ]:
# Merge data with geometry data
sf['Postcode'] = sf['Postcode'].astype(int).astype('str')
data['Postcode'] = data['Postcode'].astype(int).astype('str')
data = data.merge(sf[['Postcode', 'geometry']], left_on = 'Postcode', right_on = 'Postcode')


In [ ]:
m = folium.Map(location = [-36.9848, 143.3906], tiles = "Stamen Terrain", zoom_start = 10)
c = folium.Choropleth(
     # geoJSON 
    geo_data = geoJSON,
    # name of plot
    name ='choropleth',
    # data source
    data = data, 
    # the columns required
    columns=['Postcode','rate_2023'], 
    # this is from the geoJSON's properties
    key_on='properties.Postcode', 
    # color scheme
    fill_color='YlOrRd', 
    # Color fill nan
    nan_fill_color='white',
    # Add legend
    legend_name='Top 10 postcode region with higest growth rate'
   
)

c.add_to(m)
c.geojson.add_child(
    folium.features.GeoJsonTooltip(['Postcode'])
)

# save the map
m.save('../plots/top ten.html')
m

In [ ]:
# Read in data and drop nan
livable = pd.read_csv("../data/curated/liveable.csv")
livable = livable.dropna()
livable = livable.iloc[:,1:]

In [ ]:
# Calculate scores of each evaluation factors
livable['safety level'] = 1/(livable['Crime Count']/livable['Population'])
livable['rent/income'] = 1/((52*livable['Weekly Rental Price'])/livable['Income'])
livable = livable[['Postcode','safety level', 'rent/income','Count_Station', 'Count_Hospital', 'Count_School', 'Count_Shopping_Center']]
livable_con = livable.iloc[:,1:]


In [ ]:
# Convert all the value into 0-1
min_max_scaler = preprocessing.MinMaxScaler()
livable_con = min_max_scaler.fit_transform(livable_con)

In [ ]:
# Select safety level above the median
livable.iloc[:,1:] = livable_con
livable = livable.groupby('Postcode').mean()
livable = livable[(livable["safety level"] >= livable["safety level"].median())]

In [ ]:
# Save the liveability evaluation data
livable["Postcode"] = livable.index
livable.dropna()
livable.to_csv("../data/curated/radar.csv")

In [ ]:
import numpy as np
import pandas as pd
import shapely.geometry
import plotly.graph_objects as go

def area(row):
    '''This function takes the rows of dataframe as the input, and give area of the radar plot as output'''
    # The six evaluation factors 
    categories = ['safety level', 'rent/income', 'Count_Station', 'Count_Hospital', 
                  'Count_School', 'Count_Shopping_Center']  # fmt: skip

    fig = go.Figure()
    fig.add_trace(
        go.Scatterpolar(
        r = list(livable.iloc[row,0:6]), theta=categories, fill="toself", name = str(livable.iloc[row,6])))
    
    # Calclulate the area of the radar plot
    df = pd.concat([
        pd.DataFrame({"r": t.r, "theta": t.theta, "trace": np.full(len(t.r), t.name)})
        for t in fig.data])
    df["theta_n"] = pd.factorize(df["theta"])[0]
    df["theta_radian"] = (df["theta_n"] / (df["theta_n"].max() + 1)) * 2 * np.pi
    df["x"] = np.cos(df["theta_radian"]) * df["r"]
    df["y"] = np.sin(df["theta_radian"]) * df["r"]
    df_a = df.groupby("trace").apply(lambda d: shapely.geometry.MultiPoint(list(zip(d["x"], d["y"]))).convex_hull.area)
    
    return df_a

# Create a new list to store the area of each row
area_list = []
for row in range(0, len(livable.index)):
    result = area(row)
    radar_area = result[str(livable.iloc[row,6])]
    area_list.append(radar_area)

# Add new list into the data
livable.insert(0,"Radar Area", area_list)  
livable.to_csv("../data/curated/radar.csv")
livable

In [ ]:
# Loop through the data and find out the top
max_area = 0
max_i = 0
for i in range(0,len(area_list)):
    if float(area_list[i]) >= max_area:
        max_area = area_list[i]
        max_i = i
print(max_area)
print(max_i)
print(str(livable.iloc[53]))

In [ ]:
# Sort the data, and find out the top three area 
livable = livable.sort_values(by=['Radar Area'], ascending=False)
livable = livable.head(3)
livable.index = [0,1,2]
livable.head(5)

In [ ]:
# Convert some score for a better visulization 
livable["safety level"] = livable["safety level"]*10
livable["rent/income"] = livable["rent/income"]*100

In [ ]:
# Draw radar plot for the top 3
categories = ['Safety level', 'Affordable', 'Transportation', 'Hospital', 
                  'School', 'Shopping']

# Plot of top 1
fig = go.Figure()
fig.add_trace(
        go.Scatterpolar(
        r = list(livable.iloc[0,1:7]), theta=categories, fill="toself",
            line =  dict(color = 'gold'), name = str(livable.iloc[0,7])))
fig.update_layout(
    showlegend=True,
    title = dict(text = str(livable.iloc[0,7])),
    polar=dict(radialaxis=dict(visible=True, range=[0, 0.5])))

In [ ]:
# Plot of top 2
fig = go.Figure()
fig.add_trace(
        go.Scatterpolar(
        r = list(livable.iloc[1,1:7]), theta=categories, fill="toself",
            line =  dict(color = 'silver'), name = str(livable.iloc[1,7])))
fig.update_layout(
    showlegend=True,
    polar=dict(radialaxis=dict(visible=True, range=[0, 0.5])))

In [ ]:
# Plot of top 3
fig = go.Figure()
fig.add_trace(
        go.Scatterpolar(
        r = list(livable.iloc[2,1:7]), theta=categories, fill="toself",
            line =  dict(color = 'orange'), name = str(livable.iloc[2,7])))
fig.update_layout(
    showlegend=True,
    polar=dict(radialaxis=dict(visible=True, range=[0, 0.5])))